In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

        
        
#the data set 
data_dir = '/kaggle/input/binary-cropped-crown-of-thorns-dataset/'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
all_paths = []
all_labels = []

for label in os.listdir(data_dir):
    for image in os.listdir(data_dir+label):
        all_paths.append(data_dir+label+'/'+image)
        all_labels.append(label)

all_paths, all_labels = shuffle(all_paths, all_labels)

In [ ]:
plt.figure(figsize=(14,6))
colors = ["#0FC9F4", "#0F45F4", "#ae7cd1", "#a670cd"]
plt.pie([len([x for x in all_labels if x=='cots_crops']),
         len([x for x in all_labels if x=='notcots_crops'])],
        labels=['cots_crops','notcots_crops'],
        colors=colors, autopct='%.1f%%', explode=(0.0,0.05),
        startangle=45);

**Split the data between training and validation **

In [ ]:
x_train_paths, x_val_paths, y_train, y_val = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(x=[len([x for x in y_train if x=='cots_crops']),
               len([x for x in y_train if x=='notcots_crops']),
               len([x for x in y_val if x=='cots_crops']),
               len([x for x in y_val if x=='notcots_crops'])],
            y=['cots_crops -Train',
               'notcots_crops -Train',
               'cots_crops -Val',
               'notcots_crops -Val'],
           palette = sns.light_palette("#0FC9F4", reverse = True, n_colors = int(19 * 1.2)))
plt.show()

In [ ]:

def open_images(paths, augment=True):
    images = []
    for path in paths:
        image = load_img(path, target_size=(128,128))
        image = np.array(image)
#         if augment:
#             image = augment_image(image)
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)

In [ ]:
images = open_images(x_train_paths[150:161])
labels = y_train[50:61]
fig = plt.figure(figsize=(12, 6))
for x in range(1, 11):
    fig.add_subplot(2, 5, x)
    plt.axis('off')
    plt.title(labels[x])
    plt.imshow(images[x])
plt.show()

In [ ]:
import random

def augment_image(image):
    if random.uniform(0,1)>0.5:
        image = np.fliplr(image)
    image = Image.fromarray(np.uint8(image))
    image = ImageEnhance.Brightness(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Contrast(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Color(image).enhance(random.uniform(0.6,1.4))
    return image


def open_images(paths, augment=True):
    images = []
    for path in paths:
        image = load_img(path, target_size=(128,128))
        image = np.array(image)
        if augment:
             image = augment_image(image)
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)

images = open_images(x_train_paths[150:161])

labels = y_train[50:61]

fig = plt.figure(figsize=(12, 6))
for x in range(1, 11):
    fig.add_subplot(2, 5, x)
    plt.axis('off')
    plt.title(labels[x])
    plt.imshow(images[x])
plt.show()


In [ ]:
base_model = MobileNetV2(input_shape=(128,128, 3), weights='imagenet', include_top=False)
model = Sequential()
model.add(Input(shape=(128,128,3)))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    x_train_paths,
    validation_data=x_val_paths,
    epochs=30,
    verbose=0,
    augment=True,
)